# Initialisation
- Mount persistent libraries
- Configure protobuf

In [1]:
import sys
lib_path = '/home/jovyan/libs'
sys.path.insert(0, lib_path)

In [2]:
# necessary for whisperx to import correctly
!pip install --target=$libs_path torcheval

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 179 kB 81.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


# WhisperX Trial

In [46]:
import whisperx
import numpy as np
from datasets import load_dataset

In [4]:
device = "cpu"
batch_size = 32 # reduce if low on GPU mem
compute_type = "float32" # change to "int8" if low on GPU mem (may reduce accuracy)

model = whisperx.load_model("tiny.en", device, compute_type=compute_type)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint libs/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
tedlium = load_dataset("AudioLLMs/tedlium3_test", streaming=True)["test"]

In [22]:
def infer_tedlium3(model, data: dict, has_gpu=False) -> None:
    dtype = np.float32 if not has_gpu else np.float16
    transformed_data = data["context"]["array"].astype(dtype)
    results = model.transcribe(transformed_data)
    print(f"Transcription: {results['segments'][0]['text']}\nActual: {data['answer']}")

In [24]:
samples = []
iter_tedlium = iter(tedlium)
for i in range(3):
    samples.append(next(iter_tedlium))

In [25]:
for sample in samples:
    infer_tedlium3(model, sample)
    print("\n")

Transcription:  I'd like to share with you a discovery that I made a few months ago while writing an article for Italian Wired. I always keep my desires handy whenever I'm writing anything, but
Actual: i 'd like to share with you a discovery that i made a few months ago while writing an article for italian wired i always keep my thesaurus handy whenever i 'm writing anything but


Transcription:  I had already finished editing the piece and I realized that I had never once in my life looked up the word disabled to see what I'd find. Let me redo the entry.
Actual: i 'd already finished editing the piece and i realized that i had never once in my life looked up the word disabled to see what i 'd find let me read you the entry


Transcription:  Disables, adjectives, crippled, helpless, useless, wrecked.
Actual: disabled adjective crippled helpless useless wrecked


